In [15]:
from dotenv import load_dotenv
import os


load_dotenv(dotenv_path=".env")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = "agent-book"
os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY")
os.environ["COHERE_API_KEY"] = os.getenv("COHERE_API_KEY")

In [2]:
from langchain_community.document_loaders import GitLoader

def file_filter(file_path: str) -> bool:
    # Example filter: only load .md files
    return file_path.endswith('.md')

loader = GitLoader(
    clone_url="https://github.com/langchain-ai/langchain",
    repo_path="./langchain",
    branch="master",
    file_filter=file_filter
)

documents = loader.load()
print(len(documents))

43


In [3]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
db = Chroma.from_documents(documents, embeddings)

In [12]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template('''\
以下の文脈だけを踏まえて質問に回答してください。
                                          
文脈:"""
{context}
"""

質問: {question}
''')

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

retriever = db.as_retriever()

chain = {
    "question": RunnablePassthrough(),
    "context": retriever,
} | prompt | model | StrOutputParser()

chain.invoke("langchainの概要を教えてください。")

'LangChainは、エージェントやLLM（大規模言語モデル）を活用したアプリケーションを構築するためのフレームワークです。このフレームワークは、相互運用可能なコンポーネントやサードパーティの統合を組み合わせることで、AIアプリケーションの開発を簡素化し、技術の進化に対応できるように設計されています。\n\n主な特徴としては以下があります：\n\n- **リアルタイムデータの拡張**：多様なデータソースやシステムにLLMを簡単に接続できる。\n- **モデルの相互運用性**：異なるモデルを簡単に入れ替えられるため、最適な選択を迅速に行える。\n- **迅速なプロトタイピング**：モジュール式のアーキテクチャにより、アプリケーションの構築と反復が迅速に行える。\n- **生産準備が整った機能**：モニタリングや評価、デバッグのための統合があり、信頼性の高いアプリケーションを展開できる。\n- **活発なコミュニティとエコシステム**：多くの統合やテンプレート、コミュニティが提供するコンポーネントを活用できる。\n\nLangChainは、開発者がLLMアプリケーションを構築する際に、標準的なインターフェースを提供し、さまざまな統合を通じてアプリケーションの開発を加速します。'

In [5]:
hypothetical_prompt = ChatPromptTemplate.from_template('''\
次の質問に回答する一文を書いてください。
                                                       
質問: {question}
''')

hypothetical_chain = hypothetical_prompt | model | StrOutputParser()

hyde_rag_chain = {
    "question": RunnablePassthrough(),
    "context": hypothetical_chain | retriever,
} | prompt | model | StrOutputParser()

hyde_rag_chain.invoke("langchainの概要を教えてください。")

'LangChainは、エージェントやLLM（大規模言語モデル）を活用したアプリケーションを構築するためのフレームワークです。このフレームワークは、相互運用可能なコンポーネントやサードパーティの統合を組み合わせることで、AIアプリケーションの開発を簡素化し、技術の進化に対応できるように設計されています。\n\n主な特徴としては以下があります：\n\n- **リアルタイムデータの拡張**：多様なデータソースやシステムにLLMを簡単に接続できる。\n- **モデルの相互運用性**：異なるモデルを簡単に入れ替えられるため、最適な選択を迅速に行える。\n- **迅速なプロトタイピング**：モジュール式のアーキテクチャにより、アプリケーションの構築と反復が迅速に行える。\n- **生産準備が整った機能**：モニタリングや評価、デバッグのための組み込みサポートがあり、信頼性の高いアプリケーションを展開できる。\n- **活発なコミュニティとエコシステム**：豊富な統合やテンプレート、コミュニティが提供するコンポーネントを活用できる。\n\nLangChainは、単独で使用することもできますが、他のLangChain製品とシームレスに統合できるため、LLMアプリケーションを構築するための包括的なツールセットを提供します。'

In [7]:
from pydantic import BaseModel, Field

class QueryGnerationOutput(BaseModel):
    queries: list[str] = Field(description="検索クエリのリスト")

query_generation_prompt = ChatPromptTemplate.from_template("""\
質問に対してベクターデータベースから関連文書を検索するために、
3つの異なる検索クエリを生成してください。
距離ベースの類似性検索の限界を克服するために、
ユーザーの質問に対して複数の視点を提供することを目標としています。

質問: {question}
""")

query_generation_chain = (
    query_generation_prompt
    | model.with_structured_output(QueryGnerationOutput)
    | (lambda x: x.queries)
)

In [9]:
multi_query_rag_chain = {
    "question": RunnablePassthrough(),
    "context": query_generation_chain | retriever.map(),
} | prompt | model | StrOutputParser()

multi_query_rag_chain.invoke("langchainの概要を教えてください。")

'LangChainは、エージェントやLLM（大規模言語モデル）を活用したアプリケーションを構築するためのフレームワークです。このフレームワークは、相互運用可能なコンポーネントやサードパーティの統合を組み合わせることで、AIアプリケーションの開発を簡素化し、技術の進化に対応できるように設計されています。\n\n主な特徴としては以下があります：\n\n- **リアルタイムデータの拡張**：多様なデータソースやシステムにLLMを簡単に接続できる。\n- **モデルの相互運用性**：異なるモデルを簡単に入れ替えられるため、最適な選択を迅速に行える。\n- **迅速なプロトタイピング**：モジュール式のアーキテクチャにより、アプリケーションの構築と反復が迅速に行える。\n- **生産準備が整った機能**：モニタリングや評価、デバッグのための統合が組み込まれており、信頼性の高いアプリケーションを展開できる。\n- **活発なコミュニティとエコシステム**：多くの統合やテンプレート、コミュニティが提供するコンポーネントを活用できる。\n\nLangChainは、開発者がLLMアプリケーションを構築する際に、標準的なインターフェースを提供し、さまざまなモデルやツールとの統合を容易にします。'

In [10]:
# RAG-Fusion
from langchain_core.documents import Document

def reciprocal_rank_fusion(
    retriever_outputs: list[list[Document]],
    k: int = 60,
) -> list[str]:
    content_score_mapping = {}

    for docs in retriever_outputs:
        for rank, doc in enumerate(docs):
            content = doc.page_content
            if content not in content_score_mapping:
                content_score_mapping[content] = 0
            content_score_mapping[content] += 1 / (rank + k)

    ranked = sorted(
        content_score_mapping.items(), key=lambda x: x[1], reverse=True
    )
    return [content for content, _ in ranked]

In [14]:
rag_fusion_chain = {
    "question": RunnablePassthrough(),
    "context": query_generation_chain | retriever.map() | reciprocal_rank_fusion,
} | prompt | model | StrOutputParser()

rag_fusion_chain.invoke("langchainの概要を教えてください。")

'LangChainは、エージェントやLLM（大規模言語モデル）を活用したアプリケーションを構築するためのフレームワークです。このフレームワークは、相互運用可能なコンポーネントやサードパーティの統合を組み合わせることで、AIアプリケーションの開発を簡素化し、技術の進化に対応できるように設計されています。\n\n主な特徴としては以下があります：\n\n- **リアルタイムデータの拡張**：多様なデータソースやシステムにLLMを簡単に接続できる。\n- **モデルの相互運用性**：異なるモデルを簡単に入れ替えられる。\n- **迅速なプロトタイピング**：モジュール式のアーキテクチャにより、アプリケーションの構築と反復が迅速に行える。\n- **生産準備が整った機能**：モニタリングや評価、デバッグのための統合が組み込まれている。\n- **活発なコミュニティとエコシステム**：多くの統合やテンプレート、コミュニティが提供されている。\n\nLangChainは、開発者がLLMアプリケーションを迅速に構築できるようにするための標準インターフェースを提供し、将来的な技術の変化にも対応できる柔軟性を持っています。'

In [16]:
from typing import Any

from langchain_cohere import CohereRerank
from langchain_core.documents import Document

def rerank(inp: dict[str, Any], top_n: int = 3) -> list[Document]:
    question = inp["question"]
    documents = inp["documents"]

    cohere_reranker = CohereRerank(model="rerank-multilingual-v3.0", top_n=top_n)
    return cohere_reranker.compress_documents(documents=documents, query=question)

rerank_rag_chain = (
    {
        "question": RunnablePassthrough(),
        "documents": retriever,
    }
    | RunnablePassthrough.assign(context=rerank)
    | prompt | model | StrOutputParser()
)

rerank_rag_chain.invoke("langchainの概要を教えてください。")

'LangChainは、エージェントやLLM（大規模言語モデル）を活用したアプリケーションを構築するためのフレームワークです。このプラットフォームは、相互運用可能なコンポーネントやサードパーティの統合を組み合わせることで、AIアプリケーションの開発を簡素化し、技術の進化に対応できるように設計されています。\n\n主な特徴としては以下があります：\n\n- **リアルタイムデータの拡張**：多様なデータソースやシステムにLLMを簡単に接続できる。\n- **モデルの相互運用性**：異なるモデルを簡単に入れ替えられるため、最適な選択を迅速に行える。\n- **迅速なプロトタイピング**：モジュール式のアーキテクチャにより、アプリケーションの構築と反復が迅速に行える。\n- **生産準備が整った機能**：モニタリングや評価、デバッグのための統合があり、信頼性の高いアプリケーションを展開できる。\n- **活発なコミュニティとエコシステム**：豊富な統合やテンプレート、コミュニティが提供するコンポーネントを活用できる。\n\nLangChainは、開発者がLLMアプリケーションを構築するための標準インターフェースを提供し、さまざまなニーズに応じた柔軟な抽象化レイヤーを持っています。'

In [18]:
# 複数のRetrieverを組み合わせる
from langchain_community.retrievers import TavilySearchAPIRetriever

langchain_document_retriever = retriever.with_config(
    {"run_name": "langchain_document_retriever"}
)

web_retriever = TavilySearchAPIRetriever(k=3).with_config(
    {"run_name": "web_retriever"}
)

In [19]:
from enum import Enum

class Route(str, Enum):
    langchain_document = "langchain_document"
    web = "web"

class RouteOutput(BaseModel):
    route: Route

route_prompt = ChatPromptTemplate.from_template("""\
質問に回答するために適切なRetrieverを選択してください。
                                                
質問: {question}
""")

route_chain = (
    route_prompt
    | model.with_structured_output(RouteOutput)
    | (lambda x: x.route)
)

In [20]:
def routed_retriever(inp: dict[str, Any]) -> list[Document]:
    question = inp["question"]
    route = inp["route"]

    if route == Route.langchain_document:
        return langchain_document_retriever.invoke(question)
    elif route == Route.web:
        return web_retriever.invoke(question)
    
    raise ValueError(f"Unknown retriever: {retriever}")

route_rag_chain = (
    {
        "question": RunnablePassthrough(),
        "route": route_chain,
    }
    | RunnablePassthrough.assign(context=routed_retriever)
    | prompt | model | StrOutputParser()
)

In [21]:
route_rag_chain.invoke("langchainの概要を教えてください。")

'LangChainは、エージェントやLLM（大規模言語モデル）を活用したアプリケーションを構築するためのフレームワークです。このフレームワークは、相互運用可能なコンポーネントやサードパーティの統合を組み合わせることで、AIアプリケーションの開発を簡素化し、技術の進化に対応できるように設計されています。\n\n主な特徴としては以下があります：\n\n- **リアルタイムデータの拡張**：多様なデータソースやシステムにLLMを簡単に接続できる。\n- **モデルの相互運用性**：異なるモデルを簡単に入れ替えられるため、最適な選択を迅速に行える。\n- **迅速なプロトタイピング**：モジュール式のアーキテクチャにより、アプリケーションの構築と反復が迅速に行える。\n- **生産準備が整った機能**：モニタリングや評価、デバッグのための統合があり、信頼性の高いアプリケーションを展開できる。\n- **活発なコミュニティとエコシステム**：多くの統合やテンプレート、コミュニティが提供するコンポーネントを活用できる。\n\nLangChainは、開発者がLLMアプリケーションを構築する際に、標準的なインターフェースを提供し、さまざまなニーズに応じた柔軟な抽象化レイヤーを持っています。'

In [22]:
route_rag_chain.invoke("東京の今日の天気を教えてください。")

'東京の今日、11月13日(木)の天気は、南部では雲が多く、お昼ごろに雨が降る可能性があります。北部は晴れ間があるでしょう。最高気温は16度で、日差しが少ないため少しヒンヤリと感じられるかもしれません。暖かい服装でお過ごしください。お出かけには傘を持っていると安心です。'

In [23]:
from langchain_community.retrievers import BM25Retriever
from langchain_core.runnables import RunnableParallel

chroma_retriever = retriever.with_config(
    {"run_name": "chroma_retriever"}
)

bm25_retriever = BM25Retriever.from_documents(documents).with_config(
    {"run_name": "bm25_retriever"}
)

hybrid_retriever = (
    RunnableParallel({
        "chroma_documents": chroma_retriever,
        "bm25_documents": bm25_retriever,
    })
    | (lambda x: [x["chroma_documents"] + x["bm25_documents"]])
    | reciprocal_rank_fusion
)

In [24]:
hybrid_retriever_rag_chain = (
    {
        "question": RunnablePassthrough(),
        "context": hybrid_retriever,
    }
    | prompt | model | StrOutputParser()
)
hybrid_retriever_rag_chain.invoke("langchainの概要を教えてください。")

'LangChainは、エージェントやLLM（大規模言語モデル）を活用したアプリケーションを構築するためのフレームワークです。このフレームワークは、相互運用可能なコンポーネントやサードパーティの統合を組み合わせることで、AIアプリケーションの開発を簡素化し、技術の進化に対応できるように設計されています。\n\n主な特徴としては以下があります：\n\n- **リアルタイムデータの拡張**：多様なデータソースやシステムにLLMを簡単に接続できる。\n- **モデルの相互運用性**：異なるモデルを簡単に入れ替えられる。\n- **迅速なプロトタイピング**：モジュール式のアーキテクチャにより、アプリケーションの構築と反復が迅速に行える。\n- **生産準備が整った機能**：モニタリングや評価、デバッグのための統合が組み込まれている。\n- **活発なコミュニティとエコシステム**：多くの統合やテンプレート、コミュニティが提供されている。\n\nLangChainは、開発者がLLMアプリケーションを迅速に構築できるようにするための標準インターフェースを提供し、将来的な技術の変化にも対応できる柔軟性を持っています。'